# Script

In [1]:
import os
import json
import yaml
import numpy as np
import pandas as pd

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
import pandas as pd


# Variables
path_pdf = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\cencora_earning_reports\AmerisourceBergen-Reports-Fiscal-2023-Second-Quarter-Results-2023.pdf"

embed_model= "text-embedding-ada-002"
vectore_store_persist_directory = 'vector_store'

# Config
import sys
import os
sys.path.append(os.path.expanduser("~/.utils"))
from util_openai import get_config_dict
config = get_config_dict(env=None)

openai_api_key="242ad9b0c6d84b5fb4307061d5f05a86"
openai_api_version="2023-03-15-preview"

openai_api_base="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev"
azure_endpoint="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev"
deployment_name='gpt-35-turbo-16k'

# Load pdf
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(path_pdf)
documents = loader.load()

# Split Documents Into Chunks
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# embedding
from langchain.embeddings import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    chunk_size=1,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    )

# vector store
from langchain.vectorstores import FAISS

pages = loader.load_and_split()
vector_store = FAISS.from_documents(documents=pages, embedding=embeddings)
vector_store.save_local(vectore_store_persist_directory)

# llm
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_key=openai_api_key,
    deployment_name=deployment_name,
    model_name="text-davinci-002",
    azure_endpoint=azure_endpoint,
    openai_api_version=openai_api_version,
    temperature=0
    )

# Get vector store from local file
from langchain.vectorstores import FAISS

vector_store = FAISS.load_local(vectore_store_persist_directory, embeddings, allow_dangerous_deserialization=True,)
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2},
    return_source_documents=True,
)

# Create RetrivalQA Chain
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = retriever
)

# Create Query And Get Response
query = 'What are the year to year gain in revenue?'
result = qa.invoke({'query': query})
print(result['result'])
print('='*100)

## Using: ConversationalRetrievalChain
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(llm,retriever)
chat_history = []
query = "Is there gain or loss of revenue in this quarter?"

result = qa.invoke({'question': query, 'chat_history': chat_history})
print(result['answer'])
print('='*100)
chat_history = [(query,result['answer'])]
query = "Is there gain in revenue for U.S. Healthcare Solutions this quarter?"

result = qa.invoke({'question': query, 'chat_history': chat_history})
print(result['answer'])

C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_community\embeddings\azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev to https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev/openai.
  warnings.warn(
C:\Users\a126291\venv\py311_cam

The year-to-year gain in revenue is 7.6%.
In the second quarter of fiscal 2023, there was a decrease in revenue for International Healthcare Solutions of 0.2 percent compared to the previous fiscal year's second quarter. However, there was an increase in revenue for U.S. Healthcare Solutions of 11.3 percent compared to the same quarter in the prior fiscal year. Overall, there was a mixed result with a slight decrease in revenue for International Healthcare Solutions and an increase in revenue for U.S. Healthcare Solutions.
Based on the provided information, it is not clear whether U.S. Healthcare Solutions experienced an increase in revenue this quarter. The information only mentions a decline in revenue for International Healthcare Solutions, but does not provide specific details about the revenue of U.S. Healthcare Solutions.
